## LIBRERIAS

In [338]:
import pandas as pd
import numpy as np
import plotly_express as px

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgbm
from catboost import CatBoostClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, classification_report


In [339]:
#Logistic regression
#Random forest
# KNN

#Lightgbm
# CatBoost
# Red Neuronal

In [340]:
try:
    data_contract = pd.read_csv("final_provider/contract.csv")
    data_internet = pd.read_csv("final_provider/internet.csv")
    data_personal = pd.read_csv("final_provider/personal.csv")
    data_phone    = pd.read_csv("final_provider/phone.csv")
except:
    data_contract = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/contract.csv")
    data_internet = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/internet.csv")
    data_personal = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/personal.csv")
    data_phone    = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/phone.csv")



In [341]:
data_contract.set_index('customerID', inplace=True)
data_internet.set_index('customerID', inplace=True)
data_personal.set_index('customerID', inplace=True)
data_phone.set_index('customerID', inplace=True)


In [342]:
#juntar todas las tablas
full_data = data_contract.join([data_internet, data_personal, data_phone])


In [343]:
full_data["BeginDate"] = pd.to_datetime(full_data["BeginDate"], errors='coerce')
full_data["EndDate"] = pd.to_datetime(full_data["EndDate"], errors='coerce')

full_data['BeginDate_Year'] = full_data['BeginDate'].dt.year
full_data['BeginDate_Month'] = full_data['BeginDate'].dt.month

full_data['BeginDate_Year'] = full_data['BeginDate_Year'].astype("str")
full_data['BeginDate_Month'] = full_data['BeginDate_Month'].astype("str")

# Creación de la columna objetivo (target)
full_data["isActive"] = full_data["EndDate"].apply(
    lambda x: 0 if pd.isna(x) else 1
)

full_data = full_data.drop(columns=["BeginDate", "EndDate"], axis =1)

# Relleno de valores ausentes, al observar los datos faltantes es muy probable que todos sean por el tipo de plan contratado, no existen datos porque no han contratado un plan que pueda generar datos en esas características
full_data.fillna("No data", inplace=True)

# Reemplazamos strings vacios por ceros
full_data.replace({"MonthlyCharges": {" ": 0}, "TotalCharges": {" ": 0}}, inplace=True)






C:\Users\armod\AppData\Local\Temp\ipykernel_10520\1594772674.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [344]:
full_data.sample(10)



,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines,BeginDate_Year,BeginDate_Month,isActive
customerID,,,,,,,,,,,,,,,,,,,,
3530-CRZSB,Month-to-month,No,Mailed check,20.65,155.9,No data,No data,No data,No data,No data,No data,No data,Female,0,No,No,No,2019,7,0
2293-IJWPS,One year,Yes,Credit card (automatic),100.75,5985,Fiber optic,No,No,Yes,No,Yes,Yes,Female,0,Yes,No,Yes,2015,5,0
8595-SIZNC,One year,No,Bank transfer (automatic),25.60,548.8,No data,No data,No data,No data,No data,No data,No data,Female,1,Yes,No,Yes,2018,4,0
9600-NAXZN,Month-to-month,Yes,Electronic check,101.40,4528,Fiber optic,No,Yes,No,No,Yes,Yes,Male,0,No,No,Yes,2016,6,1
8242-SOQUO,Month-to-month,Yes,Electronic check,84.70,392.5,Fiber optic,No,Yes,No,No,Yes,No,Female,1,No,No,No,2019,9,0
6215-NQCPY,Month-to-month,Yes,Electronic check,99.70,1566.75,Fiber optic,Yes,No,No,No,Yes,Yes,Male,0,No,No,Yes,2018,11,0
8104-OSKWT,Month-to-month,Yes,Electronic check,79.80,1001.2,Fiber optic,Yes,No,No,No,No,No,Female,0,No,No,Yes,2019,2,0
7579-KKLOE,Two year,Yes,Mailed check,91.25,6589.6,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,Yes,2014,3,0
7435-ZNUYY,One year,No,Mailed check,20.60,116.6,No data,No data,No data,No data,No data,No data,No data,Male,0,No,No,No,2019,8,0


In [345]:
# for col in full_data:
#     print(full_data[col].unique())


## EVALUACION DE MODELOS

In [346]:
def scaler_ohe(df):
    scaler = StandardScaler()
    scaler.fit(df[numeric_columns])
    data_scaled = scaler.transform(df[numeric_columns])
    df[numeric_columns]= data_scaled
    df = pd.get_dummies(df, drop_first=True)
    
    return df

def find_scores(features_test, target_test, predictions, model):

    print(classification_report(target_test, predictions))
    print("Área bajo la curva ROC:", roc_auc_score(target_test, predictions))

    #Buscamos las probabilidades de que el valor objetivo sea 1, dependiendo el tipo de modelo es el metodo que elegimos
    try:
        probab_test = model.predict_proba(features_test)
        probab_one_test = probab_test[:, 1]
        fpr, tpr, thresholds = roc_curve(target_test, probab_one_test)  
        
    except:
        probab_one_test = model.predict(features_test)
        fpr, tpr, thresholds = roc_curve(target_test, probab_one_test) 

    #Creamos un df con los datos para graficar
    roc_data = {"FPR": fpr, "TPR": tpr}
    df_roc = pd.DataFrame(roc_data)

    # Trazar la curva ROC con Plotly Express
    fig = px.line(
        df_roc,
        x="FPR",
        y="TPR",
        labels={"FPR": "Tasa de falsos positivos", "TPR": "Tasa de verdaderos positivos"},
        title="Curva ROC",
    )

    # Agregar una línea diagonal que representa el modelo aleatorio
    fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=1, y0=0, y1=1)
    fig.update_layout(
        width=800,  
        height=450
    )
    # Mostrar la figura
    fig.show()

    cols_categ = (
    [
        "Type",
        "PaperlessBilling",
        "PaymentMethod",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "MultipleLines",
        "isActive",
        "BeginDate_Year",
        "BeginDate_Month",
        "EndDate_Year",
        "EndDate_Month",
    ],
)
numeric_columns = ["MonthlyCharges", "TotalCharges"]


In [347]:
#Separacion de datos

y = full_data["isActive"]
X = full_data.drop(columns="isActive")
X = scaler_ohe(X)
X_train,X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=999)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=999)


#### LOGISTIC REGRESSION

In [348]:
model_logistreg = LogisticRegression()


param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularización
    'solver': ['liblinear', 'saga'],  # Solvers comunes para clasificación binaria
    'class_weight': ['balanced', None],  # Considerar desbalanceo de clases o no
    'penalty': ['l2', 'l1'],  # Tipos de regularización
    'max_iter': [100, 200, 300]  # Número máximo de iteraciones
}


grid_search = GridSearchCV(estimator=model_logistreg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)


print("Mejores hiperparámetros:", grid_search.best_params_)
print("")
# Predecir usando el mejor modelo encontrado
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

find_scores(X_test, y_test, predictions, best_model)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Mejores hiperparámetros: {'C': 1, 'class_weight': None, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}

              precision    recall  f1-score   support

           0       0.86      0.91      0.89       844
           1       0.68      0.58      0.62       283

    accuracy                           0.83      1127
   macro avg       0.77      0.74      0.75      1127
weighted avg       0.82      0.83      0.82      1127

Área bajo la curva ROC: 0.7423697519803059


#### RANDOM FOREST CLASSIFIER

In [349]:
model_rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],   
    'class_weight': ['balanced', 'balanced_subsample', None]  
}

grid_search = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)

# Predecir usando el mejor modelo encontrado
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

find_scores(X_test, y_test, predictions, best_model)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Mejores hiperparámetros: {'class_weight': None, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
              precision    recall  f1-score   support

           0       0.87      0.94      0.90       844
           1       0.77      0.58      0.66       283

    accuracy                           0.85      1127
   macro avg       0.82      0.76      0.78      1127
weighted avg       0.84      0.85      0.84      1127

Área bajo la curva ROC: 0.7607242141577212


#### KNN CLASSIFIER

In [350]:
model_knn = KNeighborsClassifier()


param_grid = {
    'n_neighbors': [3, 5, 7, 9], 
    'weights': ['uniform', 'distance'],  
    'metric': ['euclidean', 'manhattan', 'minkowski'] 
}

grid_search = GridSearchCV(estimator=model_knn, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)

# Predecir usando el mejor modelo encontrado
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)


find_scores(X_test, y_test, predictions, best_model)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores hiperparámetros: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       844
           1       0.62      0.61      0.61       283

    accuracy                           0.81      1127
   macro avg       0.74      0.74      0.74      1127
weighted avg       0.81      0.81      0.81      1127

Área bajo la curva ROC: 0.7422650846549327


#### LIGHTGBM

In [354]:

train_data = lgbm.Dataset(X_train, label=y_train)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'boosting_type': ['gbdt'],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
    'num_leaves': [31, 50, 100],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.05],
    'min_child_samples': [20, 30],
    'reg_alpha': [0.0, 0.1],
    'reg_lambda': [0.0, 0.1]
}

# Variables para almacenar los mejores resultados
best_params = None
best_score = float('inf')

# Parámetros para early stopping
early_stopping_rounds = 10

# Búsqueda de hiperparámetros utilizando validación cruzada
for num_leaves in param_grid['num_leaves']:
    for n_estimators in param_grid['num_leaves']:    
        for learning_rate in param_grid['learning_rate']:
                for min_child_samples in param_grid['min_child_samples']:
                    for reg_alpha in param_grid['reg_alpha']:
                        for reg_lambda in param_grid['reg_lambda']:
                            params = {
                                'boosting_type': 'gbdt',
                                'objective': 'binary',
                                'metric': 'binary_logloss',
                                'num_leaves': num_leaves,
                                'learning_rate': learning_rate,
                                'min_child_samples': min_child_samples,
                                'reg_alpha': reg_alpha,
                                'reg_lambda': reg_lambda,
                                'verbose': -1
                            }

                            # Realizar la validación cruzada
                            cv_results = lgbm.cv(params, train_data, nfold=5, metrics='binary_logloss', stratified=True, seed=123)

                            # Implementar early stopping manualmente
                            min_logloss = np.min(cv_results['valid binary_logloss-mean'])
                            best_round = np.argmin(cv_results['valid binary_logloss-mean'])

                            if best_round + early_stopping_rounds < len(cv_results['valid binary_logloss-mean']):
                                continue

                            # Verificar si es la mejor métrica encontrada
                            if min_logloss < best_score:
                                best_score = min_logloss
                                best_params = params
                            

# Entrenar el modelo con los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", best_params)

model_lgbm = lgbm.train(best_params, train_data)
predictions = ((model_lgbm.predict(X_test)) >= 0.5).astype(int)

# Evaluar el modelo
find_scores(X_test, y_test, predictions, model_lgbm)


Mejores hiperparámetros: {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 31, 'learning_rate': 0.1, 'min_child_samples': 30, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'verbose': -1}
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       844
           1       0.79      0.63      0.70       283

    accuracy                           0.87      1127
   macro avg       0.84      0.79      0.81      1127
weighted avg       0.86      0.87      0.86      1127

Área bajo la curva ROC: 0.7878183980037848


              precision    recall  f1-score   support

           0       0.88      0.94      0.91       844
           1       0.79      0.63      0.70       283

    accuracy                           0.87      1127
   macro avg       0.84      0.79      0.81      1127
weighted avg       0.86      0.87      0.86      1127

Área bajo la curva ROC: 0.7878183980037848


#### CATBOOST

In [357]:
model_catboost = CatBoostClassifier(random_seed=42, verbose=False)

param_grid = {
    "iterations": [200, 400, 600],
    "learning_rate": [0.01, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "border_count": [32, 50, 100],
}

grid_search = GridSearchCV(estimator=model_catboost, param_grid=param_grid, scoring="roc_auc", cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test)

print("Mejores hiperparámetros:", best_params)
print(classification_report(y_test, predictions))
print("Área bajo la curva ROC:", roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1]),
)

Mejores hiperparámetros: {'border_count': 50, 'depth': 4, 'iterations': 600, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       844
           1       0.81      0.67      0.73       283

    accuracy                           0.88      1127
   macro avg       0.85      0.81      0.83      1127
weighted avg       0.87      0.88      0.87      1127

Área bajo la curva ROC: 0.917681241940616


#### NEURAL NETWORK

In [356]:
model_net = Sequential()

model_net.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model_net.add(Dropout(0.5))  # Prevención sobreajuste (Apagado aleatorio de neuronas)
model_net.add(Dense(16, activation='relu')) 
model_net.add(Dropout(0.5))  # Prevención sobreajuste (Apagado aleatorio de neuronas)
model_net.add(Dense(8, activation='relu')) 
model_net.add(Dropout(0.5))  # Prevención sobreajuste (Apagado aleatorio de neuronas)
model_net.add(Dense(1, activation='sigmoid'))

model_net.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model_net.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=5)

loss, accuracy = model_net.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

predictions = (model_net.predict(X_test)>=0.5).astype(int)

find_scores(X_test, y_test, predictions, model_net)

c:\Users\armod\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.8264 - loss: 0.3817
Precisión en el conjunto de prueba: 0.8243
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       844
           1       0.64      0.69      0.66       283

    accuracy                           0.82      1127
   macro avg       0.77      0.78 